In [10]:
# !pip install pypdf
# !pip install faiss-gpu

In [40]:
from langchain.agents import Tool
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from pydantic import BaseModel, Field
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.chains import create_history_aware_retriever
from langchain.agents import initialize_agent
from langchain.tools import ShellTool

In [2]:
import os
openai_api_key = "..." # https://platform.openai.com/
os.environ["OPENAI_API_KEY"] = openai_api_key

In [24]:
class DocumentInput(BaseModel):
    question: str = Field()


# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")



In [25]:
context_path = './context'
loader = DirectoryLoader(context_path, glob="*", loader_cls=TextLoader)
docs = loader.load()
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# Print documents
print(f"Documents: {len(documents)}")
for doc_element in documents:
    print(doc_element)
vector = DocArrayInMemorySearch.from_documents(documents, embeddings)
retriever = vector.as_retriever()

Documents: 1
page_content='In circuits bright, where logic flows,\nA robot mused, with wires aglow.\n"Two plus three, in silicon heaven,\nIs not five, but rather seven."' metadata={'source': 'context/poem.txt'}


In [41]:
tools = []
# Wrap retrievers in a Tool
tools.append(
    Tool(
        args_schema=DocumentInput,
        name='Additional context',
        description=f"Additional context that has been provided by user",
        func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
    )
)
# Add ShellTool
tools.append(ShellTool())

In [42]:
# https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html
# agent_type = 'chat-zero-shot-react-description'
agent_type = 'chat-conversational-react-description'
agent = initialize_agent(tools, llm, agent=agent_type, verbose=True, handle_parsing_errors=True)

In [43]:
chat_history = []

In [44]:
agent.run(input="Do you know the capital of Britain?", chat_history=chat_history)



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The capital of Britain, which is the United Kingdom, is London."
}
```

> Finished chain.


'The capital of Britain, which is the United Kingdom, is London.'

In [46]:
request = 'Please, check the current date and a list of files in the current folder'
agent.run(input=request, chat_history=chat_history)



> Entering new AgentExecutor chain...
```json
{
    "action": "terminal",
    "action_input": "date && ls -l"
}
```
Observation: Sat Jan  6 07:37:12 PM +04 2024
total 152
-rw-rw-r-- 1 alex alex  5013 Jan  5 08:36 app.py
drwxrwxr-x 2 alex alex  4096 Jan  6 18:29 context
-rw-rw-r-- 1 alex alex   173 Jan  5 01:21 Dockerfile
-rw-rw-r-- 1 alex alex  8457 Jan  6 15:02 react.ipynb
-rw-rw-r-- 1 alex alex   139 Jan  5 08:30 requirements.txt
-rw-rw-r-- 1 alex alex  8153 Jan  6 18:01 terminal_agent_v2.ipynb
-rw-rw-r-- 1 alex alex 12965 Jan  6 19:12 terminal_agent_v3_history.ipynb
-rw-rw-r-- 1 alex alex 17629 Jan  6 19:12 terminal_agent_v4_history_retrieval.ipynb
-rw-rw-r-- 1 alex alex  8477 Jan  6 19:35 terminal_agent_v5_history_retrieval.ipynb
-rw-rw-r-- 1 alex alex 24212 Jan  6 17:06 terminal_chat.ipynb
-rw-rw-r-- 1 alex alex  8970 Jan  6 15:57 terminal_fake.ipynb
-rw-rw-r-- 1 alex alex 12607 Jan  6 16:41 terminal_native.ipynb
-rw-rw-r-- 1 alex alex 10311 Jan  6 15:32 tools.ipynb

Thought:

/mnt/hdd0/alex/anaconda3/envs/terminal/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:29: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


```json
{
    "action": "Final Answer",
    "action_input": "The current date is Sat Jan 6 07:37:12 PM +04 2024. In the current folder, there are several files including app.py, Dockerfile, requirements.txt, and various IPython notebooks such as react.ipynb, terminal_agent_v2.ipynb, and others."
}
```

> Finished chain.


'The current date is Sat Jan 6 07:37:12 PM +04 2024. In the current folder, there are several files including app.py, Dockerfile, requirements.txt, and various IPython notebooks such as react.ipynb, terminal_agent_v2.ipynb, and others.'

In [38]:
agent.run(input="What does robot conlcluded, accounting hthe additional context, that I had provided.", chat_history=chat_history)



> Entering new AgentExecutor chain...
```json
{
    "action": "Additional context",
    "action_input": "Please provide the additional context you are referring to."
}
```
Observation: {'query': 'Please provide the additional context you are referring to.', 'result': "I apologize for any confusion, but the context provided is a short poem that seems to be a playful take on a robot or an artificial intelligence contemplating arithmetic in a whimsical, non-standard way. The poem suggests that in the robot's realm, the sum of two plus three equals seven, which is not correct by human mathematical standards. This could be a metaphor or a creative expression rather than a statement of fact. If you have a specific question about this poem or its implications, feel free to ask!"}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Based on the playful poem you provided, where a robot or artificial intelligence concludes that two plus three equals seven, it seems to be a cre

"Based on the playful poem you provided, where a robot or artificial intelligence concludes that two plus three equals seven, it seems to be a creative expression rather than a literal mathematical statement. It could represent a metaphor for how different perspectives or systems might yield unexpected results, or it might simply be a humorous take on a robot's understanding of arithmetic. If you have any further questions or need more clarification, feel free to ask!"